# Kubernetes Networking

In the previous lesson, you learned how to manage pods using Kubernetes workloads. Recall that workloads can control the lifecycle of pods; however, they operate in isolation from each other, and their actions are solely determined by the arguments passed to them.

> To enable seamless communication among pods and with the entire cluster, Kubernetes provides networking resources that expose the pods to each other and external services. This networking infrastructure is crucial for the proper functioning of your applications within a Kubernetes environment.

<p align=center><img src=images/K8_networking.webp width=600></p>

One important concept we didn't cover in the previous lesson is that each pod has its own unique IP address. In this lesson, we'll delve into the details of Kubernetes networking, including how these IP addresses are assigned and how pods can communicate with each other over the network.

## Hands-On: Viewing Pod Networking Information

1. Start `minikube` if you haven't already

2. Click on the following [link](https://cdn.theaicore.com/content/lessons/92324888-b4a5-41f1-bd52-f159ef357212/deployment.yaml) to access a file named `deployment.yaml`.  This file contains the configuration for a Kubernetes resource.

3. Run the correct `kubectl` command to facilitate the deployment

4. Run the following command: `kubectl get pods`, to view the description of the pods

5. Run the following command: `kubectl get pods -o wide`, to view a more detailed description of the pods

In [11]:
# deploy configuration 
!kubectl #complete command here

deployment.apps/hello-deployment created


In [12]:
# view pod information
!kubectl #complete command here

NAME                              READY   STATUS    RESTARTS   AGE
hello-deployment-fc9d6479-8dq8t   1/1     Running   0          7s
hello-deployment-fc9d6479-dhsvf   1/1     Running   0          7s
hello-deployment-fc9d6479-f88nn   1/1     Running   0          7s


> Your Pods name and IP addresses might differ when running this example. Don't worry about this!

In [13]:
# view advanced pod information
!kubectl #complete command here

NAME                              READY   STATUS    RESTARTS   AGE   IP            NODE       NOMINATED NODE   READINESS GATES
hello-deployment-fc9d6479-8dq8t   1/1     Running   0          38s   10.244.0.90   minikube   <none>           <none>
hello-deployment-fc9d6479-dhsvf   1/1     Running   0          38s   10.244.0.91   minikube   <none>           <none>
hello-deployment-fc9d6479-f88nn   1/1     Running   0          38s   10.244.0.89   minikube   <none>           <none>


As can be observed, each pod has an IP address that is only visible when the `wide` (i.e. the more detailed) output is printed.

> However, pod IP addresses in Kubernetes are not durable. Whenever an application is scaled up or down, or encounters an error and needs to be rebooted, the IP addresses disappear and need to be reassigned. This change in IP address occurs without warning. In response to this, Kubernetes utilizes *Services*.

## Services

**Services** are fundamental components that facilitate network communication within your cluster. They abstract the underlying complexities of pod management and help ensure that applications can communicate reliably and efficiently, whether they are running on the same node or distributed across a large cluster.

<p align=center><img src=images/K8_Services.webp></p>

As we've mentioned before, pod IP addresses in Kubernetes are not durable. Whenever an application scales up or down, encounters an error, or needs to be rebooted, the IP addresses assigned to pods can change unpredictably. This dynamic behavior poses a challenge for maintaining consistent connectivity within your application.

Let's consider what happens when a pod is deleted from the deployment resource:

### Hands-On

1. Observe the IP addresses from the last deployment you created

2. Delete one of the pods in the deployment resource

3. Observe the IP addresses once more

In [14]:
# get advanced information about the pods 
!kubectl #complete command here

NAME                              READY   STATUS    RESTARTS   AGE   IP            NODE       NOMINATED NODE   READINESS GATES
hello-deployment-fc9d6479-8dq8t   1/1     Running   0          64s   10.244.0.90   minikube   <none>           <none>
hello-deployment-fc9d6479-dhsvf   1/1     Running   0          64s   10.244.0.91   minikube   <none>           <none>
hello-deployment-fc9d6479-f88nn   1/1     Running   0          64s   10.244.0.89   minikube   <none>           <none>


In [15]:
# delete one of the pods
!kubectl #complete command here

pod "hello-deployment-fc9d6479-8dq8t" deleted


In [16]:
# get advanced information about the pods again
!kubectl #complete command here

NAME                              READY   STATUS    RESTARTS   AGE    IP            NODE       NOMINATED NODE   READINESS GATES
hello-deployment-fc9d6479-8nrrs   1/1     Running   0          16s    10.244.0.92   minikube   <none>           <none>
hello-deployment-fc9d6479-dhsvf   1/1     Running   0          108s   10.244.0.91   minikube   <none>           <none>
hello-deployment-fc9d6479-f88nn   1/1     Running   0          108s   10.244.0.89   minikube   <none>           <none>


As you can observe, the name and IP address of one of the pods have changed. Therefore, any attempt to point to the previous IP address will yield no result.

To address this issue, Kubernetes introduces the concept of services. Here's how it works:

- Kubernetes assigns a single virtual IP address to a specified group of pods associated with a service

- Any incoming traffic addressed to this virtual IP address is automatically routed to the corresponding set of pods, maintaining seamless connectivity

- While the set of pods linked to a service can change over time (e.g., during scaling or pod failures), the IP address of the service remains static and unchanging

This approach ensures that your applications can consistently access the desired pods without worrying about the underlying IP address changes.

### Types of Services

In Kubernetes, there are three primary types of services, each catering to specific use cases:

- *Cluster IP*: The default service type, which provides a fixed IP address within the cluster. It serves as a small load balancer, distributing traffic among the pods in the cluster.

- *Node Port*: Similar to Cluster IP but also creates a port on each node, allowing external traffic to reach the service, which then redirects it to the corresponding pods

- *Load Balancer*: Exposes the service externally using a cloud provider's load balancer, making it accessible from the internet. It leverages cloud resources to route network traffic to the pods.

Click on this [link](https://kubernetes.io/docs/concepts/services-networking/service/#publishing-services-service-types) for additional details. 

### Cluster IP

> Cluster IP is a built-in networking resource that assigns a single virtual IP address to a group of pods within the cluster. This virtual IP address provides an internal entry point for applications and services running within the cluster, allowing them to communicate seamlessly.

In the upcoming example, we will run a practical demo to illustrate how Kubernetes Services work, specifically using the `ClusterIP` service type. We'll connect the service to pods managed by a `Deployment` resource, showcasing the stability and reliability of services in a dynamic environment.

``` yaml
apiVersion: v1
kind: Service
metadata:
  name: hello-service
spec:
  ports:
    - port: 80
      targetPort: 80
  selector:
    app: hello
```

Observe the `spec` field, this is the specification section where you define the configuration for the Service :

- `ports`: Here, you specify the ports that the Service should listen on and how they should be directed to the pods:

  - `port: 80`: This line specifies that the Service should listen on port 80. This is the port that clients will use to connect to the Service.
  - `targetPort: 80`: It indicates that incoming traffic on port 80 should be directed to pods' port 80. This is where the pods are listening for requests.

- `selector`: This part defines a label selector, which is used to determine which pods the Service should route traffic to. In this case, it selects pods with the label `app: hello`. This means that any pod with the label `app` set to `hello` will be part of the set of pods targeted by this Service.

For demonstration, we will run a single Pod, which will run an Ubuntu container inside the cluster. Then we will connect to that cluster and the service afterwards:

``` yaml
apiVersion: v1
kind: Pod
metadata:
  name: hello-pod
  labels:
    app: hello
spec:
  containers:
    - name: nginx-container
      image: nginx:1.14.2
```

> In Kubernetes, you can define both different types of resources within the same `YAML` file using `---` as a separator. This approach allows you to manage the application's deployment and its associated service configuration together, making it more convenient for deploying and exposing applications in a single, structured `YAML` document.

For the example above, this would look like this:


```yaml
apiVersion: v1
kind: Service
metadata:
  name: hello-service
spec:
  ports:
    - port: 80
      targetPort: 80
  selector:
    app: hello
---
apiVersion: v1
kind: Pod
metadata:
  name: hello-pod
  labels:
    app: hello
spec:
  containers:
    - name: nginx-container
      image: nginx:1.14.2
```

### Hands-On: Demonstrating Kubernetes Services

1. Create a Kubernetes Service and a Deployment resource using the provided configuration. Save this configuration in a file named `service-deployment.yaml`. Separate the two resources within the file using three dashes (`---`).

2. Apply the `YAML` configuration to your Kubernetes cluster. This will create both the Service and Deployment

3. Run the necessary `kubectl` command to view a list of all the pods and services

4. Use the necessary `kubectl` command to view details about the IP address that the Service points to

5. Use the necessary `kubectl` command to observe the IP addresses of all pods

6. Compare the IP addresses of the individual pods with the endpoints pointed to by the Service,

7. Delete one of the pods associated with the Deployment resource

8. Compare the new addresses with the updated endpoints pointed to by the Service. Determine if the Service is still correctly pointing to the remaining IP addresses.

In [19]:
#Run both resources.
!kubectl 

service/hello-service created
pod/hello-pod created


In [20]:
# Run a command to check all pods and services
!kubectl

NAME                                  READY   STATUS    RESTARTS   AGE
pod/hello-deployment-fc9d6479-8nrrs   1/1     Running   0          3m24s
pod/hello-deployment-fc9d6479-dhsvf   1/1     Running   0          4m56s
pod/hello-deployment-fc9d6479-f88nn   1/1     Running   0          4m56s
pod/hello-pod                         1/1     Running   0          16s

NAME                    TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
service/hello-service   ClusterIP   10.110.92.113   <none>        80/TCP    16s
service/kubernetes      ClusterIP   10.96.0.1       <none>        443/TCP   47h


As shown above, the service `service/hello`, has been created. This is a `ClusterIP` type service, meaning it's accessible only from within the cluster. The service has the internal IP address `10.110.92.113` and listens on port 80 for incoming TCP traffic. The `EXTERNAL-IP` field is marked as `<none>` indicating it's not externally exposed.

In [21]:
#Run a command to describe the IP addresses pointed to by the service:
!kubectl 

Name:              hello-service
Namespace:         default
Labels:            <none>
Annotations:       <none>
Selector:          app=hello
Type:              ClusterIP
IP Family Policy:  SingleStack
IP Families:       IPv4
IP:                10.110.92.113
IPs:               10.110.92.113
Port:              <unset>  80/TCP
TargetPort:        80/TCP
Endpoints:         10.244.0.89:80,10.244.0.91:80,10.244.0.92:80
Session Affinity:  None
Events:            <none>


In [22]:
# Run a command to observe all the IP addresses of all pods.
!kubectl 

NAME                              READY   STATUS    RESTARTS   AGE     IP            NODE       NOMINATED NODE   READINESS GATES
hello-deployment-fc9d6479-8nrrs   1/1     Running   0          4m4s    10.244.0.92   minikube   <none>           <none>
hello-deployment-fc9d6479-dhsvf   1/1     Running   0          5m36s   10.244.0.91   minikube   <none>           <none>
hello-deployment-fc9d6479-f88nn   1/1     Running   0          5m36s   10.244.0.89   minikube   <none>           <none>
hello-pod                         1/1     Running   0          56s     10.244.0.93   minikube   <none>           <none>


> Don't worry here if you have a different IP address locally, just make sure the IP addresses of the Pods match the endpoints of the `ClusterIP`.

In [23]:
# Delete one of the pods corresponding to the `Deployment` resource
!kubectl 

pod "hello-deployment-fc9d6479-8nrrs" deleted


In [24]:
# Compare the new addresses with the new endpoints pointed by the `Service`
!kubectl 
!kubectl 

NAME                              READY   STATUS    RESTARTS   AGE     IP            NODE       NOMINATED NODE   READINESS GATES
hello-deployment-fc9d6479-dhsvf   1/1     Running   0          6m17s   10.244.0.91   minikube   <none>           <none>
hello-deployment-fc9d6479-f88nn   1/1     Running   0          6m17s   10.244.0.89   minikube   <none>           <none>
hello-deployment-fc9d6479-z6fs6   1/1     Running   0          18s     10.244.0.94   minikube   <none>           <none>
hello-pod                         1/1     Running   0          97s     10.244.0.93   minikube   <none>           <none>
Name:              hello-service
Namespace:         default
Labels:            <none>
Annotations:       <none>
Selector:          app=hello
Type:              ClusterIP
IP Family Policy:  SingleStack
IP Families:       IPv4
IP:                10.110.92.113
IPs:               10.110.92.113
Port:              <unset>  80/TCP
TargetPort:        80/TCP
Endpoints:         10.244.0.89:80,10.24

As can be observed, the IP addresses have changed, and the service is pointing to the newly created IP address. Thus, as mentioned before, Service will create the necessary routes to obtain the correct IP addresses, even when the pod has been rebooted.

### Node Port

> Node Port is a fundamental networking concept in Kubernetes. It opens a specific port on each node within the cluster, acting as a gateway to your applications. 

These `NodePort` services allow external access to your applications, making them accessible from outside the cluster. This means that clients, including web browsers and other applications, can connect to your services through the exposed port on any node, providing a way to interact with your applications from the external world. 

The process of creating a `NodePort` service closely resembles that of a `ClusterIP`. However, there is a crucial difference – you explicitly specify the service type as `NodePort`. Let's explore an example `YAML` configuration to illustrate this concept further:

``` yaml
apiVersion: v1
kind: Service
metadata:
  name: hello
spec:
  type: NodePort
  ports:
  - port: 8080
    targetPort: 8080
    nodePort: 30000
  selector:
    role: hello
```

#### Hands-On: `NodePort`

1. Create a `YAML` file named `nodeport-example.yaml`, that defines the three resources: an `ubuntu` Pod, a `hello-deployment` Deployment (from the previous hands-on), and the `hello` `NodePort` Service provided above. The `ubuntu` Pod should have the following specifications:

```yaml
# Define the Pod
apiVersion: v1
kind: Pod
metadata:
  name: ubuntu-pod
spec:
  containers:
    - name: ubuntu-container
      image: ubuntu
      command: ["sleep", "infinity"]
```
Make sure to separate each of the three resources in the `YAML` using `---`.

2. Apply the `YAML` file using `kubectl` to create the resources in your cluster

3. Using the necessary `kubectl` command view all the resources that you have created

4. Using the necessary `kubectl` command to obtain information about the `NodePort` service 

In [26]:
# create resources
!kubectl 

pod/ubuntu-pod created
deployment.apps/hello-deployment unchanged
service/hello-service-nodeport created


In [27]:
# Show all the services you created
!kubectl 

NAME                                  READY   STATUS    RESTARTS   AGE
pod/hello-deployment-fc9d6479-dhsvf   1/1     Running   0          12m
pod/hello-deployment-fc9d6479-f88nn   1/1     Running   0          12m
pod/hello-deployment-fc9d6479-z6fs6   1/1     Running   0          6m23s
pod/hello-pod                         1/1     Running   0          7m42s
pod/ubuntu-pod                        1/1     Running   0          15s

NAME                               READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hello-deployment   3/3     3            3           12m

NAME                             TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
service/hello-service            ClusterIP   10.110.92.113   <none>        80/TCP         7m42s
service/hello-service-nodeport   NodePort    10.103.16.66    <none>        80:30287/TCP   15s
service/kubernetes               ClusterIP   10.96.0.1       <none>        443/TCP        47h


In [29]:
# Obtain information about the node
!kubectl 

Name:                     hello-service-nodeport
Namespace:                default
Labels:                   <none>
Annotations:              <none>
Selector:                 app=hello
Type:                     NodePort
IP Family Policy:         SingleStack
IP Families:              IPv4
IP:                       10.103.16.66
IPs:                      10.103.16.66
Port:                     <unset>  80/TCP
TargetPort:               80/TCP
NodePort:                 <unset>  30287/TCP
Endpoints:                10.244.0.89:80,10.244.0.91:80,10.244.0.94:80
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


### Load Balancer

> Load Balancers provide an efficient way to distribute incoming requests across multiple nodes within your Kubernetes cluster. They are commonly available as a service provided by cloud providers, but you can also explicitly specify them in your Kubernetes configuration.

While they share some similarities with Node Ports, Load Balancers stand out in their ability to intelligently route traffic. They create a flexible, single IP address that can connect to any available node, providing a unified access point for your applications. In contrast, Node Ports assign individual IP addresses to each node, necessitating direct connections to specific nodes.

Load Balancers offer an additional advantage in that they automatically configure the exposed port for you. To create a `LoadBalancer` service, all you need to do is specify `LoadBalancer` as the value for `spec.type` in your configuration. This simplicity means that the Load Balancer service takes care of the port configuration, making it easier for you to set up external access to your services without having to manually manage port assignments. For example, if you create a `LoadBalancer` service that listens on port 80, external traffic directed to this service will enter the cluster via an automatically assigned port on the external load balancer. This port assignment is handled by Kubernetes and may vary depending on the cloud provider or load balancer implementation.

Let's look at a hands-on example that will guide you through the process of creating and using a `LoadBalancer` service in Kubernetes:

### Hands-On: Creating a `LoadBalancer` Service

1. Create a `YAML` file, let's name it `loadbalancer-example.yaml`, that defines the three resources: an `ubuntu` Pod, a `hello` Deployment, and a `hello` `LoadBalancer` Service. Your `LoadBalancer` should have the following specification:

``` yaml
apiVersion: v1
kind: Service
metadata:
  name: hello
spec:
  type: LoadBalancer
  ports:
  - port: 8080
    targetPort: 8080
  selector:
    role: hello
```

For the `ubuntu` Pod and `hello` Deployment keep the same examples as before.

2. Apply the `loadbalancer-example.yaml` file to create the `LoadBalancer` service in your cluster:

In [31]:
!kubectl 

pod/ubuntu-pod unchanged
deployment.apps/hello-deployment unchanged
service/hello-service-loadbalancer created


To check the status of your service and obtain information about the external IP address allocated by the `LoadBalancer`, run:

In [43]:
!kubectl get svc

NAME                         TYPE           CLUSTER-IP       EXTERNAL-IP   PORT(S)          AGE
hello-service                ClusterIP      10.110.92.113    <none>        80/TCP           32m
hello-service-loadbalancer   LoadBalancer   10.111.248.104   <pending>     8080:30766/TCP   19m
hello-service-nodeport       NodePort       10.103.16.66     <none>        80:30287/TCP     25m
kubernetes                   ClusterIP      10.96.0.1        <none>        443/TCP          2d


This command will display information about the `hello` service, including the external IP address (under the `EXTERNAL-IP` column) once it's allocated. Initially, the `EXTERNAL-IP` field might be in a `<pending>` state until the Load Balancer is fully provisioned.

Once the external IP address is assigned (it might take a few minutes), you can access your application using this IP address. You can use a web browser or any HTTP client to connect to your application using the external IP and port `8080`.

## Ingress

> An *Ingress* is a powerful object that simplifies external access to services within a cluster. It serves as a critical component for managing incoming HTTP and HTTPS traffic and offers features like load balancing and SSL termination. Essentially, Ingress provides a way to control and route external web traffic to specific services or applications running within a Kubernetes cluster.

![](./images/ingress-one-service.png)

Some features of Ingress include:

- **External Access**: Ingress exposes HTTP and HTTPS services from outside the cluster, providing a unified entry point to access services within

- **Load Balancing**: It can act as a load balancer, efficiently distributing incoming requests to the appropriate services within the cluster

- **SSL Termination**: When an Ingress handles SSL termination, it acts as the endpoint for encrypted traffic. The Ingress decrypts the incoming SSL-encrypted data (SSL is a popular encryption mechanism), exposing the unencrypted content to the services within the cluster. This allows for secure communication from external clients to your applications running in the cluster without the need for those applications to manage SSL decryption themselves. It simplifies the process of handling secure web traffic within the cluster and provides an added layer of security.

It's important to note that an Ingress controller must be installed and configured to work with Kubernetes. Various Ingress controllers are available, each offering different capabilities. You can explore the available options in the [official Kubernetes documentation](https://kubernetes.io/docs/concepts/services-networking/ingress-controllers/).

To enable Ingress in Minikube, you can run the following command:

In [56]:
!minikube addons enable ingress

💡  After the addon is enabled, please run "minikube tunnel" and your ingress resources would be available at "127.0.0.1"
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
    ▪ Using image k8s.gcr.io/ingress-nginx/controller:v1.0.0-beta.3
🔎  Verifying ingress addon...
🌟  The 'ingress' addon is enabled


You can verify that the `Ingress` controller is working correctly by checking the status of its pods:

In [34]:
!kubectl get pods -n ingress-nginx

NAME                                        READY   STATUS      RESTARTS      AGE
ingress-nginx-admission-create-dv2nn        0/1     Completed   0             18h
ingress-nginx-admission-patch-hxjtf         0/1     Completed   1             18h
ingress-nginx-controller-7799c6795f-b7zkl   1/1     Running     1 (66m ago)   18h


The `Ingress` controller typically deploys a `nginx` controller along with a `LoadBalancer` or `NodePort`, depending on the underlying platform. Consider the following code, and observe the output:

In [35]:
!kubectl -n ingress-nginx get svc

NAME                                 TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)                      AGE
ingress-nginx-controller             NodePort    10.102.200.194   <none>        80:31325/TCP,443:31254/TCP   18h
ingress-nginx-controller-admission   ClusterIP   10.111.152.166   <none>        443/TCP                      18h


### Defining `Ingress` Resources

Let's have a look at an example of defining `Ingress` resources in Kubernetes, using `YAML` configuration:

``` yaml

apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: minimal-ingress
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /
spec:
  rules:
  - http:
      paths:
      - path: /testpath
        pathType: Prefix
        backend:
          service:
            name: test
            port:
              number: 80
```

This Ingress `YAML` configuration defines a Kubernetes `Ingress` resource, which is used to manage external access to services. The resource is named `minimal-ingress` and includes an annotation, `nginx.ingress.kubernetes.io/rewrite-target: /`, which configures an NGINX-based `Ingress` controller to rewrite the URL path to `/`. The annotation's instruction to rewrite the URL path to `/` means that when an incoming request matches this `Ingress`, the part of the URL path specified in the request will be replaced with `/`. This is useful for situations where you want to route requests with specific paths to a service but without changing the path itself in the final destination.

The Ingress specification (`spec`) contains rules for handling incoming traffic. In this case, there is one rule for HTTP traffic (`http`) with a path rule that matches requests starting with `/testpath`. When such requests are received, they are directed to a Kubernetes Service named `test` on port 80. This configuration essentially routes incoming requests with the `/testpath` prefix to the `test` Service while rewriting the URL path to `/`.

## Clean Up

Before wrapping up this lesson, let's make sure we clean up all the resources we have been provisioning this lesson. Run the following commands to make sure everything we provision is deleted:

In [ ]:
!kubectl delete -f service-deployment.yaml
!kubectl delete -f nodeport-example.yaml
!kubectl delete -f loadbalancer-example.yaml

## Key Takeaways

- Kubernetes offers various networking resources to facilitate communication between pods, services, and external clients
- `ClusterIP` is the default service type in Kubernetes. It assigns a single virtual IP address to a group of pods, providing internal communication within the cluster.
- `NodePort` creates a port on each node, exposing services to external traffic. It allows connections to applications through browsers or other clients.
- `LoadBalancer` distributes incoming traffic across nodes, providing a uniform entry point for external access. It also simplifies SSL termination.
- `Ingress` manages external access to services within the cluster, supporting HTTP and HTTPS traffic. Ingress controllers, like `nginx`, handle traffic routing and load balancing.